In [5]:
import pandas as pd
import numpy as np
import ast
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import pickle as pk

In [7]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv') 

**Knowing the Data**

In [9]:
movies.sample(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
3026,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.americangirl.com/movie/,8359,"[{""id"": 8057, ""name"": ""cincinnati""}, {""id"": 18...",en,Kit Kittredge: An American Girl,The Great Depression hits home for nine year o...,6.27141,"[{""name"": ""New Line Cinema"", ""id"": 12}, {""name...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2008-07-02,0,101.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Kit Kittredge, a resourceful young girl, helps...",Kit Kittredge: An American Girl,6.4,34
1392,0,"[{""id"": 35, ""name"": ""Comedy""}]",NaN,9957,"[{""id"": 1480, ""name"": ""baseball""}, {""id"": 4735...",en,The Benchwarmers,A trio of guys try and make up for missed oppo...,6.74556,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2006-04-07,0,80.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Get off the bench and get into the game.,The Benchwarmers,5.2,242


In [11]:
movies.shape

(4803, 20)

In [10]:
credits.sample(5)

,movie_id,title,cast,crew
1041,82675,Taken 2,"[{""cast_id"": 11, ""character"": ""Bryan Mills"", ""...","[{""credit_id"": ""598121ff92514151ae00694f"", ""de..."
3732,47452,Love and Death on Long Island,"[{""cast_id"": 1002, ""character"": ""Giles De'Ath""...","[{""credit_id"": ""52fe4734c3a36847f8128ceb"", ""de..."
1929,9903,The Hoax,"[{""cast_id"": 26, ""character"": ""Clifford Irving...","[{""credit_id"": ""52fe4544c3a36847f80c4907"", ""de..."
1252,1887,Marie Antoinette,"[{""cast_id"": 19, ""character"": ""Marie Antoinett...","[{""credit_id"": ""52fe431ec3a36847f803b8d9"", ""de..."
3910,223485,Slow West,"[{""cast_id"": 1, ""character"": ""Silas"", ""credit_...","[{""credit_id"": ""55a3d405925141297700569d"", ""de..."


In [12]:
movies = movies.merge(credits,on='title')

**Below mentioned columns will be droped as they don't make much sense**
- budget
- homepage
- id
- original_language
- original_title
- popularity
- production_comapny
- production_countries
- release-date(not sure)

In [13]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [14]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [16]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   title     4809 non-null   object
 2   overview  4806 non-null   object
 3   genres    4809 non-null   object
 4   keywords  4809 non-null   object
 5   cast      4809 non-null   object
 6   crew      4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 263.1+ KB


In [17]:
movies.dropna(inplace=True)

In [19]:
movies.duplicated().sum()

0

In [20]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L 

In [21]:
movies['genres'] = movies['genres'].apply(convert)
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [22]:
movies['keywords'] = movies['keywords'].apply(convert)
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
movies['cast'] = movies['cast'].apply(convert)
movies.head()

In [29]:
movies['cast'] = movies['cast'].apply(lambda x:x[0:3])

In [32]:
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

In [33]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

In [34]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [35]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
3445,10925,The Return of the Living Dead,When a bumbling pair of employees at a medical...,"[Comedy, Horror]","[female nudity, crematorium, nudity, punk, com...","[Clu Gulager, James Karen, Don Calfa]",[Dan O'Bannon]
2201,10710,Envy,A man becomes increasingly jealous of his frie...,[Comedy],"[jealousy, inventor, neighbor, best friend, mi...","[Ben Stiller, Jack Black, Rachel Weisz]",[Barry Levinson]
3766,6537,The Orphanage,A woman brings her family back to her childhoo...,"[Horror, Drama, Thriller]","[schizophrenia, suppressed past, wife]","[Belén Rueda, Fernando Cayo, Roger Príncep]",[Juan Antonio Bayona]
2422,41382,Lottery Ticket,Kevin Carson is a young man living in the proj...,[Comedy],[],"[Shad Moss, Brandon T. Jackson, Naturi Naughton]",[Erik White]
1084,9381,Babylon A.D.,In Babylon A.D Vin Diesel stars as a veteran-t...,"[Action, Adventure, Science Fiction, Thriller]","[submarine, baby, secret, future, mercenary, p...","[Vin Diesel, Michelle Yeoh, Mélanie Thierry]",[Mathieu Kassovitz]


In [36]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [37]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [39]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
2115,10073,Date Movie,Spoof of romantic comedies which focuses on a ...,[Comedy],"[date, diary, parents-in-law, bridegroom, wedd...","[AlysonHannigan, AdamCampbell, SophieMonk]","[JasonFriedberg, AaronSeltzer]"
3985,10226,High Tension,Alexia travels with her friend Marie to spend ...,"[Horror, Thriller, Mystery]","[massmurder, insanity, gore, survival, farmhou...","[CéciledeFrance, Maïwenn, PhilippeNahon]",[AlexandreAja]
970,1248,Hannibal Rising,"The story of the early, murderous roots of the...","[Crime, Drama, Thriller]","[winter, psychopath, horror, serialkiller]","[GaspardUlliel, AaranThomas, GongLi]",[PeterWebber]
2181,9842,School for Scoundrels,"A young guy short on luck, enrolls in a class ...","[Comedy, Drama, Romance]","[date, competition, lovesickness, mentor, traf...","[BillyBobThornton, JonHeder, JacindaBarrett]",[ToddPhillips]
2918,206563,Trash,"Set in Brazil, three kids who make a discovery...","[Adventure, Crime, Drama, Thriller]","[brazilian, drama, thriller]","[RicksonTevez, EduardoLuis, GabrielWeinstein]",[StephenDaldry]


In [40]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [41]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [42]:
new_data = movies.drop(columns=['overview','genres','keywords','cast','crew'])

In [44]:
new_data['tags'] = new_data['tags'].apply(lambda x: " ".join(x))
new_data.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [45]:
cv = CountVectorizer(max_features=5000,stop_words='english')

In [47]:
vector = cv.fit_transform(new_data['tags']).toarray()

In [48]:
vector.shape

(4806, 5000)

In [49]:
similarity = cosine_similarity(vector)

In [50]:
similarity

array([[1.        , 0.08964215, 0.06071767, ..., 0.02519763, 0.0277885 ,
        0.        ],
       [0.08964215, 1.        , 0.06350006, ..., 0.02635231, 0.        ,
        0.        ],
       [0.06071767, 0.06350006, 1.        , ..., 0.02677398, 0.        ,
        0.        ],
       ...,
       [0.02519763, 0.02635231, 0.02677398, ..., 1.        , 0.07352146,
        0.04774099],
       [0.0277885 , 0.        , 0.        , ..., 0.07352146, 1.        ,
        0.05264981],
       [0.        , 0.        , 0.        , ..., 0.04774099, 0.05264981,
        1.        ]])

In [ ]:
new_data[new_data['title'] == 'The Lego Movie'].index[0]

In [51]:
def recommend(movie):
    index = new_data[new_data['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_data.iloc[i[0]].title)
        

In [54]:
recommend('Avatar')

Titan A.E.
Small Soldiers
Ender's Game
Aliens vs Predator: Requiem
Independence Day


In [56]:
pk.dump(new_data,open('movie_list.pkl','wb'))
pk.dump(similarity,open('similarity.pkl','wb'))